<a href="https://colab.research.google.com/github/knshkp/chatbot/blob/main/ownchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import tensorflow as tf 
import numpy as np
import json
import random 
import tflearn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "location",\r\n         "patterns": 

In [5]:
with open('intents.json')as json_data:
  intents=json.load(json_data)

In [6]:
words=[]
classes=[]
documents=[]
ignore=[]
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w=nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [7]:
words=[stemmer.stem(w.lower())for w in words if w not in ignore]
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))
print(len(documents),"document")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)


31 document
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
58 unique stemmed words ["'s", '?', 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [8]:
training=[]
output=[]
output_empty=[0]*len(classes)
for doc in documents:
  bag=[]
  pattern_words=doc[0]
  pattern_words=[stemmer.stem(word.lower())for word in pattern_words]
  for w in words:
    bag.append(1)if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [9]:

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=10, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 2249  | total loss: 0.00541 | time: 1.261s
| Adam | epoch: 010 | loss: 0.00541 - acc: 1.0000 -- iter: 1792/1798
Training Step: 2250  | total loss: 0.00556 | time: 1.265s
| Adam | epoch: 010 | loss: 0.00556 - acc: 1.0000 -- iter: 1798/1798
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open("training_data","wb"))

In [13]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [14]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [15]:

model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [16]:
def clean_up_sentence(sentence):
  
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [17]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
  
    results = model.predict([bow(sentence, words)])[0]
   
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
   
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    if results:
        
        while results:
            for i in intents['intents']:
             
                if i['tag'] == results[0][0]:
                   
                    return print(random.choice(i['responses']))

            results.pop(0)

In [18]:
classify('What are you hours of operation?')

[('hours', 0.99922395)]

In [19]:
response('What are you hours of operation?')

Our hours are 9am-9pm every day


In [20]:
response('Do you accept Credit Card?')

We accept VISA, Mastercard and AMEX


In [21]:
response('Where can we locate you?')

We are situated at the intersection of London Alley and Bridge Avenue


In [22]:
response('That is helpful')

My pleasure


In [23]:

response('Bye')

Bye! Come back again soon.


In [27]:
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        return print(random.choice(i['responses']))

            results.pop(0)

In [28]:
response('Can you please let me know the delivery options?')

We are situated at the intersection of London Alley and Bridge Avenue


In [29]:
response('What is menu for today?')

Our speciality for today is Chicken Tikka
